In [1]:
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install ace_tools


import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install ace_tools
!{sys.executable} -m pip install gurobipy

import numpy as np
from itertools import product
import pprint
import copy
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import math
import statistics
import time
import sys

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
import time
import statistics
epsilon = 1e-6


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip


In [2]:

def utility(x_r, w_r, gamma):
    if gamma == 1:
        return w_r * np.log(x_r)
    else:
        return (w_r / (1 - gamma)) * x_r**(1 - gamma)

def measure_time(func):
    def wrapper(*args, **kwargs):
        execution_times = []
        results = []

        for _ in range(3):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)  # Wywołanie funkcji
            end_time = time.perf_counter()

            execution_times.append((end_time - start_time) * 1000)
            results.append(result)

        time_mean = statistics.mean(execution_times)  # Obliczenie średniego czasu
        time_std_dev = statistics.stdev(execution_times)  # Obliczenie odchylenia standardowego
        result_mean = statistics.mean(results)       # Obliczenie średniego wyniku

        return time_mean, time_std_dev, result_mean
    return wrapper

def measure_time_solver(func):
    def wrapper(*args, **kwargs):
        execution_times = []
        results = []

        for _ in range(3):
            result_val, result_time = func(*args, **kwargs)  # Wywołanie funkcji

            execution_times.append(result_time * 1000)
            results.append(result_val)

        time_mean = statistics.mean(execution_times)  # Obliczenie średniego czasu
        time_std_dev = statistics.stdev(execution_times)  # Obliczenie odchylenia standardowego
        result_mean = statistics.mean(results)       # Obliczenie średniego wyniku

        return time_mean, time_std_dev, result_mean
    return wrapper


In [3]:
def compute_initial_strategies_for_single_link(link_index, routing_matrix, capacity, weights, path_weights, gamma):
    """
    Compute the allocation s_lr for a specific link.

    Parameters:
        link_index: Index of the link (l).
        routing_matrix: np.array (L x R), where a[l][r] = 1 if flow r uses link l, else 0.
        capacity: Capacity of the specific link (c_l).
        weights: List of flow weights (w_r).
        path_weights: List of path weights (b_r), either uniform or path-length-based.
        gamma: Isoelastic parameter.

    Returns:
        strategies: List of s_lr values for the specific link (l), where s_lr is the allocation for each flow r.
    """
    flows = routing_matrix.shape[1]
    strategies = np.zeros(flows)

    # Calculate the denominator: sum of (b_j w_j)^(1/gamma) for all flows using the link
    denominator = sum(
        routing_matrix[link_index][j] * (path_weights[j] * weights[j])**(1/gamma)
        for j in range(flows)
    )

    for r in range(flows):  # Iterate over flows
        if routing_matrix[link_index][r] == 1:  # Only allocate if flow r uses the link
            # Numerator: (b_r w_r)^(1/gamma) for flow r
            numerator = capacity * (path_weights[r] * weights[r])**(1/gamma)
            # strategies for flow r on the specific link
            strategies[r] = numerator / denominator

    return strategies


def compute_strategy_for_single_link_in_iteration(link_index, routing_matrix, capacity, weights, path_weights, gamma, non_saturated_flows, previous_strategies):
    """
    Compute the allocation s_lr for a specific link in an iteration.
    """
    flows = routing_matrix.shape[1]
    strategies = np.zeros(flows)
    saturated_flows = set(range(flows)) - non_saturated_flows

    for r in saturated_flows:
        if routing_matrix[link_index][r] == 1:
            flow_capacity = min([previous_strategies[i][r] for i in range(routing_matrix.shape[0]) if routing_matrix[i][r] == 1])
            strategies[r] = flow_capacity
            capacity -= flow_capacity

    # Calculate the denominator: sum of (b_j w_j)^(1/gamma) for all flows using the link
    denominator = sum(
        routing_matrix[link_index][j] * (path_weights[j] * weights[j])**(1/gamma)
        for j in non_saturated_flows
    )

    for r in non_saturated_flows:  # Iterate over flows
        if routing_matrix[link_index][r] == 1:  # Only allocate if flow r uses the link
            # Numerator: (b_r w_r)^(1/gamma) for flow r
            numerator = capacity * (path_weights[r] * weights[r])**(1/gamma)
            # strategies for flow r on the specific link
            strategies[r] = numerator / denominator

    return strategies

In [4]:
@measure_time
def capacity_allocation_game(links, capacities, routing_matrix, weights, gamma=1):
    num_flows = len(weights)

    strategies = [compute_initial_strategies_for_single_link(l, routing_matrix, capacities[l], weights, weights, gamma) for l in range(links)]
    non_saturated_flows = set(range(num_flows))

    links_that_run_out_of_capacity = [
        l for l in range(links)
        if math.isclose(
            sum(
                min([strategies[i][r] for i in range(links) if routing_matrix[i][r] == 1])
                for r in range(num_flows)
                if routing_matrix[l][r] == 1
            ),
            capacities[l],
            rel_tol=epsilon
        )
    ]
    n = 2
    while n <= links:
        iteration_saturated_flows = set([r for r in range(num_flows) if any(routing_matrix[l][r] == 1 for l in links_that_run_out_of_capacity)])
        non_saturated_flows = non_saturated_flows - iteration_saturated_flows
        new_strategies = [compute_strategy_for_single_link_in_iteration(l, routing_matrix, capacities[l], weights, weights, gamma, non_saturated_flows, strategies) for l in range(links)]
        strategies = new_strategies
        links_that_run_out_of_capacity = [
            l for l in range(links)
            if math.isclose(
                sum(
                    min([strategies[i][r] for i in range(links) if routing_matrix[i][r] == 1])
                    for r in range(num_flows)
                    if routing_matrix[l][r] == 1
                ),
                capacities[l],
                rel_tol=epsilon
            )
        ]
        n += 1
        if len(non_saturated_flows) == 0:
            break

    network_utility = sum(utility(min([strategies[l][r] for l in range(links) if routing_matrix[l][r] == 1]), weights[r], gamma) for r in range(num_flows))


    return network_utility

In [5]:
@measure_time_solver
def optimize_with_gurobi(capacities, routing_matrix, weights, gamma):
    num_links = len(capacities)
    num_flows = len(weights)
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()
        with gp.Model(env=env) as model:
            # Create the model

            # Decision variables: allocation for each flow on each link
            x = model.addVars(num_links, num_flows, lb=0, name="x")

            # Objective: Maximize total utility (reformulated for Gurobi linear/quadratic constraints)
            if gamma == 1:
                aux_vars = model.addVars(num_flows, lb=0, name="aux")
                aux_min_vars = model.addVars(num_flows, lb=0, name="aux_min")
                aux_log_vars = model.addVars(num_flows, lb=1e-6, name="aux_log")
                aux_vars_prod = model.addVars(num_flows, lb=0, name="aux_prod")

                for r in range(num_flows):
                    model.addConstr(aux_min_vars[r] == gp.min_([x[l, r] for l in range(num_links) if routing_matrix[l, r] == 1]),
                                    name=f"min_alloc_{r}")

                for r in range(num_flows):
                    model.addConstr(aux_log_vars[r] == aux_min_vars[r] + 1e-6,
                                    name=f"log_alloc_{r}")

                for r in range(num_flows):
                    model.addGenConstrLog(aux_log_vars[r], aux_vars[r], name=f"aux_constraint_{r}")

                for r in range(num_flows):
                    model.addConstr(aux_vars_prod[r] == aux_vars[r] * weights[r],
                                    name=f"prod_alloc_{r}")

                objective = gp.quicksum(aux_vars_prod[r]
                                        for r in range(num_flows))
                model.setObjective(objective, GRB.MAXIMIZE)


            else:
                aux_vars = model.addVars(num_flows, lb=0, name="aux")
                aux_min_vars = model.addVars(num_flows, lb=0, name="aux_min")

                for r in range(num_flows):
                    model.addConstr(aux_min_vars[r] == gp.min_([x[l, r] for l in range(num_links) if routing_matrix[l, r] == 1]),
                                    name=f"min_alloc_{r}")

                for r in range(num_flows):
                    model.addConstr(aux_vars[r] == (aux_min_vars[r] + 1e-6)**(1-gamma) , name=f"aux_constraint_{r}")
                objective = gp.quicksum(weights[r] / (1 - gamma) * aux_vars[r]
                                        for r in range(num_flows))
                model.setObjective(objective, GRB.MAXIMIZE)

            # Constraints
            for l in range(num_links):
                # Capacity constraint: sum of allocations on link l <= capacity
                model.addConstr(gp.quicksum(x[l, r] for r in range(num_flows) if routing_matrix[l, r] == 1) <= capacities[l],
                                name=f"capacity_{l}")

            for r in range(num_flows):
                # Flow constraint: sum of allocations across links for flow r <= flow demand
                model.addConstr(gp.quicksum(x[l, r] for l in range(num_links) if routing_matrix[l, r] == 1) >= 0,
                                name=f"non_negativity_{r}")

            # Optimize the model
            model.optimize()

            if model.status == GRB.OPTIMAL:
                # Extract the solution data
                objective_value = model.ObjVal
                time_to_solve = model.Runtime
                return objective_value, time_to_solve
            else:
                objective_value = model.ObjVal
                time_to_solve = model.Runtime
                return objective_value, time_to_solve

In [6]:
def generate_scaled_positive_normal(n, dispersion, loc=1.0, seed=None):
    """
    Generuje n pozytywnych wartości z rozkładu normalnego,
    zastępuje wartości negatywne małymi dodatnimi liczbami,
    a następnie skaluje je, aby suma wynosiła 1.

    :param n: Liczba wartości do wygenerowania
    :param dispersion: Odchylenie standardowe rozkładu normalnego
    :param loc: Średnia rozkładu normalnego
    :param seed: Opcjonalny seed dla powtarzalności
    :return: NumPy array z n skalowanymi, dodatnimi wartościami
    """
    if seed is not None:
        np.random.seed(seed)
    values = np.random.normal(loc=loc, scale=dispersion, size=n)
    # Zastąp wartości negatywne małymi dodatnimi liczbami
    values = np.where(values > 0, values, 0.01)
    scaled_values = values / np.sum(values)
    return scaled_values


def generate_capacities(n, dispersion, seed=None):
    """
    Generuje n wartości z rozkładu normalnego,
    zastępuje wartości mniejsze niż 1 wartością 1.

    :param n: Liczba wartości do wygenerowania
    :param dispersion: Odchylenie standardowe rozkładu normalnego
    :param seed: Opcjonalny seed dla powtarzalności
    :return: NumPy array z n pojemnościami >=1
    """
    if seed is not None:
        np.random.seed(seed)
    values = np.random.normal(loc=10.0, scale=dispersion, size=n)
    # Zastąp wartości mniejsze niż 1 wartością 1
    values = np.where(values >= 1, values, 1.0)
    return values

In [7]:
def generate_routing_matrix(num_links, num_flows, sparsity=0.5, seed=None):
    """
    Generuje losową macierz routingu o wymiarach (num_links x num_flows),
    zapewniając, że każdy przepływ korzysta z co najmniej jednego połączenia
    oraz każde połączenie jest używane przez co najmniej jeden przepływ.

    :param num_links: Liczba połączeń
    :param num_flows: Liczba przepływów
    :param sparsity: Prawdopodobieństwo, że dana komórka będzie wynosić 1
    :param seed: Opcjonalny seed dla powtarzalności
    :return: NumPy array o wymiarach (num_links x num_flows)
    """
    if seed is not None:
        np.random.seed(seed)
    routing_matrix = np.random.rand(num_links, num_flows) < sparsity
    routing_matrix = routing_matrix.astype(int)

    # Zapewnij, że każdy przepływ korzysta z co najmniej jednego połączenia
    for flow in range(num_flows):
        if not routing_matrix[:, flow].any():
            link = np.random.randint(0, num_links)
            routing_matrix[link, flow] = 1

    # Zapewnij, że każde połączenie jest używane przez co najmniej jeden przepływ
    for link in range(num_links):
        if not routing_matrix[link, :].any():
            flow = np.random.randint(0, num_flows)
            routing_matrix[link, flow] = 1
    return routing_matrix

In [8]:
def generate_problem_instance(
    links,
    flows,
    capacity_dispersion,
    weight_dispersion,
    gamma,
    routing_seed=None,
    capacity_seed=None,
    weight_seed=None
):
    """
    Generuje instancję problemu z zadanymi parametrami.

    :param links: Liczba połączeń
    :param flows: Liczba przepływów
    :param capacity_dispersion: Odchylenie standardowe dla pojemności
    :param weight_dispersion: Odchylenie standardowe dla wag
    :param gamma: Parametr funkcji celu
    :param routing_seed: Seed dla macierzy routingu
    :param capacity_seed: Seed dla pojemności
    :param weight_seed: Seed dla wag
    :return: Słownik zawierający wszystkie elementy instancji
    """
    capacities = generate_capacities(links, capacity_dispersion, seed=capacity_seed)
    weights = generate_scaled_positive_normal(flows, weight_dispersion, seed=weight_seed)
    routing_matrix = generate_routing_matrix(links, flows, seed=routing_seed)

    instance = {
        'links': links,
        'flows': flows,
        'capacities': capacities.tolist(),
        'weights': weights.tolist(),
        'routing_matrix': routing_matrix.tolist(),
        'gamma': gamma
    }

    return instance

In [9]:
def generate_test_instances():
    """
    Generuje wszystkie instancje testowe, zmieniając jeden parametr na raz.
    Dla każdego parametru badane są 3 różne wartości.

    :return: Lista słowników z instancjami problemu
    """
    # Definicja wartości bazowych
    base_params = {
        'gamma': 0.5,
        'weight_dispersion': 0.1,
        'links': 3,
        'flows': 3,
        'capacity_dispersion': 1
    }

    # Seedy dla powtarzalności
    base_seed = 420692137
    np.random.seed(base_seed)

    # Generowanie bazowej instancji
    base_instance = generate_problem_instance(
        links=base_params['links'],
        flows=base_params['flows'],
        capacity_dispersion=base_params['capacity_dispersion'],
        weight_dispersion=base_params['weight_dispersion'],
        gamma=base_params['gamma'],
        routing_seed=base_seed,
        capacity_seed=base_seed + 1,
        weight_seed=base_seed + 2
    )

    test_instances = []

    # Definicja możliwych wartości dla każdego parametru (3 wartości)
    param_variations = {
        'gamma': [0.3, 0.5, 0.7, 1, 2],
        'weight_dispersion': [0, 0.1, 1, 3, 5],
        'links': [2, 3, 4, 6, 10],
        'flows': [2, 3, 4, 6, 10],
        'capacity_dispersion': [0.05, 0.1, 0.2, 0.5, 1]
    }

    # Iteruj przez każdy parametr
    for param, values in param_variations.items():
        for value in values:
            # Utwórz kopię bazowej instancji
            new_instance = copy.deepcopy(base_instance)
            # Zmieniony parametr
            new_instance['gamma'] = base_params['gamma']  # default
            new_instance['weight_dispersion'] = base_params['weight_dispersion']
            new_instance['capacity_dispersion'] = base_params['capacity_dispersion']
            new_instance['links'] = base_params['links']
            new_instance['flows'] = base_params['flows']

            # Modyfikacja w zależności od parametru
            if param == 'gamma':
                new_instance['gamma'] = value
            elif param == 'weight_dispersion':
                new_instance['weight_dispersion'] = value
                # Regeneruj weights
                new_weights = generate_scaled_positive_normal(
                    base_params['flows'],
                    value,
                    seed=base_seed + 2
                )
                new_instance['weights'] = new_weights
            elif param == 'capacity_dispersion':
                new_instance['capacity_dispersion'] = value
                # Regeneruj capacities
                new_capacities = generate_capacities(
                    base_params['links'],
                    value,
                    seed=base_seed + 1
                )
                new_instance['capacities'] = new_capacities
            elif param == 'links':
                new_instance['links'] = value
                # Regeneruj capacities, weights, routing_matrix
                new_capacities = generate_capacities(
                    value,
                    base_params['capacity_dispersion'],
                    seed=base_seed + 1
                )
                new_weights = generate_scaled_positive_normal(
                    base_params['flows'],
                    base_params['weight_dispersion'],
                    seed=base_seed + 2
                )
                new_routing_matrix = generate_routing_matrix(
                    value,
                    base_params['flows'],
                    seed=base_seed
                )
                new_instance['capacities'] = new_capacities
                new_instance['weights'] = new_weights
                new_instance['routing_matrix'] = new_routing_matrix
            elif param == 'flows':
                new_instance['flows'] = value
                # Regeneruj capacities, weights, routing_matrix
                new_capacities = generate_capacities(
                    base_params['links'],
                    base_params['capacity_dispersion'],
                    seed=base_seed + 1
                )
                new_weights = generate_scaled_positive_normal(
                    value,
                    base_params['weight_dispersion'],
                    seed=base_seed + 2
                )
                new_routing_matrix = generate_routing_matrix(
                    base_params['links'],
                    value,
                    seed=base_seed
                )
                new_instance['capacities'] = new_capacities
                new_instance['weights'] = new_weights
                new_instance['routing_matrix'] = new_routing_matrix

            # Dodaj informację o zmienionym parametrze
            test_instances.append({
                'changed_parameter': param,
                'changed_value': value,
                'instance': new_instance
            })

    return test_instances

In [10]:
# Generowanie instancji testowych
test_instances = generate_test_instances()
results = {
    'gamma': [],
    'weight_dispersion': [],
    'links': [],
    'flows': [],
    'capacity_dispersion': [],

    'capacities': [],
    'routing_matrix': [],
    'weights': [],
    
    'time_mean': [],
    'time_std_dev': [],
    'result_mean': [],
    'time_mean_gurobi': [],
    'time_std_dev_gurobi': [],
    'result_mean_gurobi': []
}

# Wyświetlenie przykładowych instancji
for idx, test in enumerate(test_instances, start=1):
    time_mean, time_std_dev, result_mean = capacity_allocation_game(test["instance"]["links"],test["instance"]["capacities"],np.array(test["instance"]["routing_matrix"]),test["instance"]["weights"],test["instance"]["gamma"])
    time_mean_gurobi, time_std_dev_gurobi, result_mean_gurobi = optimize_with_gurobi(test["instance"]["capacities"],np.array(test["instance"]["routing_matrix"]),test["instance"]["weights"],test["instance"]["gamma"])
    results['gamma'].append(test['instance']['gamma'])
    results['weight_dispersion'].append(test['instance']['weight_dispersion'])
    results['links'].append(test['instance']['links'])
    results['flows'].append(test['instance']['flows'])
    results['capacity_dispersion'].append(test['instance']['capacity_dispersion'])
    
    results['capacities'].append(test["instance"]["capacities"])
    results['routing_matrix'].append(test["instance"]["routing_matrix"])
    results['weights'].append(test["instance"]["weights"])
    
    results['time_mean'].append(time_mean)
    results['time_std_dev'].append(time_std_dev)
    results['result_mean'].append(result_mean)
    results['time_mean_gurobi'].append(time_mean_gurobi)
    results['time_std_dev_gurobi'].append(time_std_dev_gurobi)
    results['result_mean_gurobi'].append(result_mean_gurobi)

df = pd.DataFrame(results)
df.to_csv('results.csv', index=False)